In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!ls /content/drive/My\ Drive/


'Colab Notebooks'
'Document from N!T!SH'
'Empowering Pharma with Data Insights.gdoc'
 IMG_20170203_175255766.jpg
'Moto x4'
'New Version (3) (1).docx'
'NITISH ALL CONTACTS'
 steam_reviews.csv
 Ticket.pdf
 Video
'what can we learn from the above case of Wells Fargo.gsheet'


In [3]:
file_path = "/content/drive/My Drive/steam_reviews.csv"


In [4]:
!pip install modin[ray]  # Install Modin with Ray backend
!pip install ray  # Ensure Ray is installed too


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.2 MB/s eta 0:00:00


In [5]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import ray
import time

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Using pandas to read the file

In [ ]:
import pandas as pd
import time

file_path = "/content/drive/My Drive/steam_reviews.csv"

start_time = time.time()
df_pandas = pd.read_csv(file_path)
pandas_time = time.time() - start_time

print(f"Pandas read time: {pandas_time:.2f} seconds")

Using dask to read the file

In [6]:
import dask.dataframe as dd
import time

file_path = "/content/drive/My Drive/steam_reviews.csv"

start_time = time.time()
df_dask = dd.read_csv(file_path, blocksize="100MB")  # Load in 100MB chunks
end_time = time.time()

print(f"Dask Read Time: {end_time - start_time:.2f} seconds")
print(df_dask.head())  # ✅ Removed .compute()


Dask Read Time: 0.78 seconds
   Unnamed: 0  app_id                  app_name  review_id  language  \
0           0  292030  The Witcher 3: Wild Hunt   85185598  schinese   
1           1  292030  The Witcher 3: Wild Hunt   85185250  schinese   
2           2  292030  The Witcher 3: Wild Hunt   85185111  schinese   
3           3  292030  The Witcher 3: Wild Hunt   85184605   english   
4           4  292030  The Witcher 3: Wild Hunt   85184287  schinese   

                                              review  timestamp_created  \
0                            不玩此生遗憾，RPG游戏里的天花板，太吸引人了         1611381629   
1                                 拔DIAO无情打桩机--杰洛特!!!         1611381030   
2                                              巫师3NB         1611380800   
3  One of the best RPG's of all time, worthy of a...         1611379970   
4                                                 大作         1611379427   

   timestamp_updated  recommended  votes_helpful  ...  steam_purchase  \
0         1611

Using modin to read the file

In [7]:
import os
import ray
import modin.pandas as mpd
import time


os.environ["MODIN_MEMORY"] = "4000000000"

ray.init(ignore_reinit_error=True)

file_path = "/content/drive/My Drive/steam_reviews.csv"

start_time = time.time()

df_modin = mpd.read_csv(file_path, nrows=500000)

end_time = time.time()

print(f"Modin Read Time: {end_time - start_time:.2f} seconds")
print(df_modin.head())


2025-02-05 18:54:18,413	INFO worker.py:1841 -- Started a local Ray instance.


Modin Read Time: 13.17 seconds
   Unnamed: 0  app_id                  app_name  review_id  language  \
0           0  292030  The Witcher 3: Wild Hunt   85185598  schinese   
1           1  292030  The Witcher 3: Wild Hunt   85185250  schinese   
2           2  292030  The Witcher 3: Wild Hunt   85185111  schinese   
3           3  292030  The Witcher 3: Wild Hunt   85184605   english   
4           4  292030  The Witcher 3: Wild Hunt   85184287  schinese   

                                              review  timestamp_created  \
0                            不玩此生遗憾，RPG游戏里的天花板，太吸引人了         1611381629   
1                                 拔DIAO无情打桩机--杰洛特!!!         1611381030   
2                                              巫师3NB         1611380800   
3  One of the best RPG's of all time, worthy of a...         1611379970   
4                                                 大作         1611379427   

   timestamp_updated  recommended  votes_helpful  ...  steam_purchase  \
0         16

Using ray to read the file

In [8]:
import ray
import time

if not ray.is_initialized():
    ray.init(ignore_reinit_error=True)

start_time = time.time()
df_ray = ray.data.read_csv("/content/drive/My Drive/steam_reviews.csv")
end_time = time.time()

print(f"Ray Read Time: {end_time - start_time:.2f} seconds")
print(df_ray.schema())  # Show detected schema


2025-02-05 18:54:47,361	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2025-02-05_18-54-09_729010_1289/logs/ray-data
2025-02-05 18:54:47,364	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadCSV]


Ray Read Time: 2.10 seconds


Running 0: 0.00 row [00:00, ? row/s]

- ReadCSV->SplitBlocks(4) 1: 0.00 row [00:00, ? row/s]

2025-02-05 18:54:48,300	ERROR streaming_executor_state.py:485 -- An exception was raised from a task of operator "ReadCSV->SplitBlocks(4)". Dataset execution will now abort. To ignore this exception and continue, set DataContext.max_errored_blocks.
2025-02-05 18:54:48,393	ERROR exceptions.py:73 -- Exception occurred in Ray Data or Ray Core internal code. If you continue to see this error, please open an issue on the Ray project GitHub page with the full stack trace below: https://github.com/ray-project/ray/issues/new/choose
2025-02-05 18:54:48,398	ERROR exceptions.py:81 -- Full stack trace:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/ray/data/exceptions.py", line 49, in handle_trace
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ray/data/_internal/plan.py", line 429, in execute_to_iterator
    bundle_iter = itertools.chain([next(gen)], gen)
                                   ^^^^^^^^^


RayTaskError(ValueError): [36mray::ReadCSV->SplitBlocks(4)()[39m (pid=2367, ip=172.28.0.12)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "pyarrow/ipc.pxi", line 705, in pyarrow.lib.RecordBatchReader.read_next_batch
  File "pyarrow/error.pxi", line 92, in pyarrow.lib.check_status
pyarrow.lib.ArrowInvalid: CSV parser got out of sync with chunker. This can mean the data file contains cell values spanning multiple lines; please consider enabling the option 'newlines_in_values'.

The above exception was the direct cause of the following exception:

[36mray::ReadCSV->SplitBlocks(4)()[39m (pid=2367, ip=172.28.0.12)
  File "/usr/local/lib/python3.11/dist-packages/ray/data/_internal/execution/operators/map_operator.py", line 507, in _map_task
    for b_out in map_transformer.apply_transform(iter(blocks), ctx):
  File "/usr/local/lib/python3.11/dist-packages/ray/data/_internal/execution/operators/map_transformer.py", line 451, in __call__
    for block in blocks:
  File "/usr/local/lib/python

In [13]:
import pandas as pd

pandas_time = "Crashed"
dask_time = 0.78
modin_time = 13.17
ray_time = 2.1

# Create a Pandas DataFrame for clear presentation
df_results = pd.DataFrame({
    "Method": ["Pandas", "Dask", "Modin", "Ray"],
    "Read Time (sec)": [pandas_time, dask_time, modin_time, ray_time],
    "Memory Usage": ["High", "Low", "Medium", "Low"],
    "Handles Large Files?": ["No", "Yes", "No", "Yes"],
    "Notes": [
        "Crashed due to memory limit",
        "Best choice, handles chunks",
        "Faster than Pandas but consumes lot of memory, not stable in colab",
        "Best for extremely large datasets. Requires more setup, may not be necessary for 3GB files.g"
    ]
})

print(df_results)


   Method Read Time (sec) Memory Usage Handles Large Files?  \
0  Pandas         Crashed         High                   No   
1    Dask            0.78          Low                  Yes   
2   Modin           13.17       Medium                   No   
3     Ray             2.1          Low                  Yes   

                                               Notes  
0                        Crashed due to memory limit  
1                        Best choice, handles chunks  
2  Faster than Pandas but consumes lot of memory,...  
3  Best for extremely large datasets. Requires mo...  


In [18]:
import dask.dataframe as dd

df_dask = dd.read_csv(file_path, blocksize="100MB")

def clean_column_names(df):
    df.columns = df.columns.str.replace(r'[^a-zA-Z0-9_]', '', regex=True)  # Remove special characters
    df.columns = df.columns.str.strip()  # Remove spaces
    return df

df_dask = clean_column_names(df_dask)

df_dask.head()




,Unnamed0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,...,steam_purchase,received_for_free,written_during_early_access,authorsteamid,authornum_games_owned,authornum_reviews,authorplaytime_forever,authorplaytime_last_two_weeks,authorplaytime_at_review,authorlast_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,1611381629,1611381629,True,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1.611343e+09
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,1611381030,1611381030,True,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1.611386e+09
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,1611380800,1611380800,True,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1.611384e+09
3,3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,1611379427,1611379427,True,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,1.610788e+09


In [21]:
import yaml

#Define schema with expected column names and separator
schema = {
    "separator": "|",  # Define the separator for the output file
    "columns": list(df_dask.columns)  # Getting column names from Dask
}

# Saving schema to a YAML file
schema_file = "schema.yaml"
with open(schema_file, "w") as file:
    yaml.dump(schema, file, default_flow_style=False)

print(f"Schema file '{schema_file}' created successfully!")


Schema file 'schema.yaml' created successfully!


In [35]:
df_dask = dd.read_csv(file_path, blocksize="100MB", on_bad_lines='skip', quoting=3)


In [36]:
df_dask = dd.read_csv(
    file_path,
    blocksize="100MB",
    on_bad_lines='skip',  # Skips problematic rows
    low_memory=False,  # Ensures consistent dtype detection
    encoding="utf-8",  # Prevents character encoding issues
    sep=",",  # Explicitly setting CSV separator
    quotechar='"',  # Ensures text within quotes is read correctly
    dtype=str  # Forces ALL columns to be read as string (safe approach)
)

print("✅ CSV successfully loaded with all columns as strings for consistency!")



✅ CSV successfully loaded with all columns as strings for consistency!


In [38]:
# Load schema from YAML
with open(schema_file, "r") as file:
    schema = yaml.safe_load(file)

# Function to validate schema without using `.compute()`
def validate_schema_dask(df, schema):
    # Get column names directly from Dask DataFrame
    dask_columns = list(df.columns)

    # Check column count
    if len(dask_columns) != len(schema["columns"]):
        print("❌ Column count mismatch!")
        return False

    # Check column names
    if dask_columns != schema["columns"]:
        print("❌ Column names mismatch!")
        return False

    print("✅ Schema validation passed successfully!")
    return True

# Apply schema validation to Dask DataFrame
validate_schema_dask(df_dask, schema)


❌ Column names mismatch!


False

In [39]:
print("📌 Columns in Dask DataFrame:", list(df_dask.columns))
print("📌 Columns in YAML Schema:", schema["columns"])


📌 Columns in Dask DataFrame: ['Unnamed: 0', 'app_id', 'app_name', 'review_id', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'recommended', 'votes_helpful', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access', 'author.steamid', 'author.num_games_owned', 'author.num_reviews', 'author.playtime_forever', 'author.playtime_last_two_weeks', 'author.playtime_at_review', 'author.last_played']
📌 Columns in YAML Schema: ['Unnamed0', 'app_id', 'app_name', 'review_id', 'language', 'review', 'timestamp_created', 'timestamp_updated', 'recommended', 'votes_helpful', 'votes_funny', 'weighted_vote_score', 'comment_count', 'steam_purchase', 'received_for_free', 'written_during_early_access', 'authorsteamid', 'authornum_games_owned', 'authornum_reviews', 'authorplaytime_forever', 'authorplaytime_last_two_weeks', 'authorplaytime_at_review', 'authorlast_played']


In [40]:
# Force column names to match schema
df_dask = df_dask.rename(columns=dict(zip(list(df_dask.columns), schema["columns"])))

print("✅ Columns renamed to match schema!")


✅ Columns renamed to match schema!


In [41]:
validate_schema_dask(df_dask, schema)


✅ Schema validation passed successfully!


True

In [46]:
df_dask_cleaned = df_dask.dropna()  # Removes rows with missing values
df_dask_cleaned = df_dask_cleaned[df_dask_cleaned.applymap(lambda x: isinstance(x, str))]  # Removes non-string values

print("✅ Cleaned dataset, now trying to save...")



✅ Cleaned dataset, now trying to save...


In [48]:
df_dask_cleaned = df_dask.dropna()  # Removes rows with missing values


In [49]:
df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.astype(str))


In [50]:
df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.applymap(lambda x: x.replace("\n", " ") if isinstance(x, str) else x))


In [4]:
# Re-load the dataset using Dask if needed
if "df_dask" not in locals():
    import dask.dataframe as dd
    file_path = "/content/drive/My Drive/steam_reviews.csv"  # Adjust the path if needed
    df_dask = dd.read_csv(file_path)

# Apply cleaning steps again
df_dask_cleaned = df_dask.dropna()
df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.astype(str))
df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.applymap(lambda x: x.replace("\n", " ") if isinstance(x, str) else x))

print("✅ `df_dask_cleaned` recreated successfully!")


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
<ipython-input-4-330bc2d3c98c>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.applymap(lambda x: x.replace("\n", " ") if isinstance(x, str) else x))


✅ `df_dask_cleaned` recreated successfully!


In [7]:
import dask.dataframe as dd

# Ensure df_dask_cleaned is properly defined
if "df_dask_cleaned" not in locals():
    print("🔄 Reloading dataset...")
    file_path = "/content/drive/My Drive/steam_reviews.csv"
    df_dask = dd.read_csv(file_path, blocksize="50MB", dtype=str, assume_missing=True)

    df_dask_cleaned = df_dask.dropna()  # Remove missing values
    df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.astype(str))  # Ensure all are strings
    df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.applymap(lambda x: x.replace("\n", " ") if isinstance(x, str) else x))  # Remove newlines

# Remove rows that might have unclosed strings
def filter_valid_rows(df):
    try:
        df.to_csv("test.csv", index=False)  # Try writing to a test file
        return df
    except:
        return None  # Skip problematic rows

df_dask_cleaned = df_dask_cleaned.map_partitions(filter_valid_rows)

print("✅ Successfully filtered out bad rows.")


✅ Successfully filtered out bad rows.


In [9]:
import dask.dataframe as dd

file_path = "/content/drive/My Drive/steam_reviews.csv"

df_dask = dd.read_csv(
    file_path,
    blocksize="50MB",  # Read in small chunks to avoid memory crashes
    dtype=str,  # Treat everything as a string to avoid conversion errors
    encoding="utf-8",
    assume_missing=True,  # Helps with missing values
    on_bad_lines="skip",  # Skips problematic rows
    low_memory=False  # Forces Pandas to handle large data better
)

print("✅ Successfully loaded CSV safely!")


✅ Successfully loaded CSV safely!


In [10]:
# Remove rows with missing values
df_dask_cleaned = df_dask.dropna()

# Convert everything to string to avoid dtype errors
df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.astype(str))

# Remove unclosed strings and new lines
df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.applymap(lambda x: x.replace("\n", " ") if isinstance(x, str) else x))

print("✅ Successfully cleaned the dataset!")


✅ Successfully cleaned the dataset!


<ipython-input-10-12af860b5390>:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.applymap(lambda x: x.replace("\n", " ") if isinstance(x, str) else x))


In [12]:
import dask.dataframe as dd

# Define cleaned file path
sample_file_path = "cleaned_steam_reviews_sample.gz"

# Save in small chunks
df_dask_cleaned.to_csv(
    sample_file_path,
    sep="|",
    index=False,
    compression="gzip",
    single_file=False,  # This ensures it is written in multiple smaller files
    encoding="utf-8",
)

print(f"✅ Cleaned dataset saved successfully in chunks at '{sample_file_path}'")



ParserError: Error tokenizing data. C error: EOF inside string starting at row 115752

In [4]:
import dask.dataframe as dd

# Reload dataset
file_path = "/content/drive/My Drive/steam_reviews.csv"

df_dask = dd.read_csv(
    file_path,
    blocksize="50MB",  # Process in small chunks to prevent crashes
    dtype=str,  # Treat everything as string to avoid type mismatches
    encoding="utf-8",
    assume_missing=True,
    on_bad_lines="skip",
    low_memory=False
)

# ✅ Fix: Use .map() instead of .applymap()
df_dask_cleaned = df_dask.dropna()  # Remove missing values
df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.astype(str))  # Ensure all data is treated as strings
df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.map(lambda x: x.replace("\n", " ") if isinstance(x, str) else x))  # Remove newlines

print("✅ Dataset reloaded and cleaned successfully!")


✅ Dataset reloaded and cleaned successfully!


In [7]:
import dask.dataframe as dd

file_path = "/content/drive/My Drive/steam_reviews.csv"  # Ensure this path is correct

# Load the file again
df_dask = dd.read_csv(file_path, dtype=str, low_memory=False)

print("✅ Dask dataframe reloaded successfully!")


✅ Dask dataframe reloaded successfully!


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [16]:
# Remove missing values
df_dask_cleaned = df_dask.dropna()

# Convert all columns to string type to prevent dtype issues
df_dask_cleaned = df_dask_cleaned.map_partitions(lambda df: df.astype(str))

# Remove newlines to prevent formatting errors
df_dask_cleaned = df_dask_cleaned.map_partitions(
    lambda df: df.map(lambda x: x.replace("\n", " ") if isinstance(x, str) else x)
)

print("✅ df_dask_cleaned created successfully!")


✅ df_dask_cleaned created successfully!


In [25]:
import os
import dask.dataframe as dd

# Define the output file path (replace with your actual file path)
output_file = "cleaned_steam_reviews_.gz"

# Get the file size in bytes
file_size = os.path.getsize(output_file)

# Load the cleaned dataset using Dask
df_dask_cleaned = dd.read_csv(output_file, sep="|", compression="gzip", assume_missing=True)

# Get the number of columns (doesn't need computation)
num_cols = len(df_dask_cleaned.columns)

# Estimate the total number of rows using Dask optimizations
num_rows = df_dask_cleaned.map_partitions(len).sum().compute()

# Print the summary
print(f"Total number of rows: {num_rows}")
print(f"Total number of columns: {num_cols}")
print(f"File size: {file_size} bytes")


FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_steam_reviews_.gz'